In [2]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import json
import joblib

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize


Importing plotly failed. Interactive plots will not work.


In [3]:
with open("prospects.json", encoding="utf-8") as f:
    prospects_raw = json.load(f)
df_prospects = pd.DataFrame.from_dict(prospects_raw, orient="index")

with open("vagas.json", encoding="utf-8") as f:
    vagas_raw = json.load(f)
df_vagas = pd.DataFrame.from_dict(vagas_raw, orient="index")

with open("applicants.json", encoding="utf-8") as f:
    applicants_raw = json.load(f)
df_applicants = pd.DataFrame.from_dict(applicants_raw, orient="index")

print(df_prospects.shape, df_vagas.shape,df_applicants.shape)



(14222, 3) (14081, 3) (42482, 7)


In [4]:
print(df_applicants.head())

                                           infos_basicas  \
31000  {'telefone_recado': '', 'telefone': '(11) 9704...   
31001  {'telefone_recado': '', 'telefone': '(11) 9372...   
31002  {'telefone_recado': '', 'telefone': '(11) 9239...   
31003  {'telefone_recado': '', 'telefone': '(11) 9810...   
31004  {'telefone_recado': '', 'telefone': '(11) 9251...   

                                    informacoes_pessoais  \
31000  {'data_aceite': 'Cadastro anterior ao registro...   
31001  {'data_aceite': 'Cadastro anterior ao registro...   
31002  {'data_aceite': 'Cadastro anterior ao registro...   
31003  {'data_aceite': 'Cadastro anterior ao registro...   
31004  {'data_aceite': 'Cadastro anterior ao registro...   

                               informacoes_profissionais  \
31000  {'titulo_profissional': '', 'area_atuacao': ''...   
31001  {'titulo_profissional': 'Analista Administrati...   
31002  {'titulo_profissional': 'Administrativo | Fina...   
31003  {'titulo_profissional': 'Área a

In [5]:
print(df_prospects.head())

                                                 titulo modalidade  \
4530                                CONSULTOR CONTROL M              
4531  2021-2607395-PeopleSoft Application Engine-Dom...              
4532                                                                 
4533  2021-2605708-Microfocus Application Life Cycle...              
4534  2021-2605711-Microfocus QTP - UFT Automation T...              

                                              prospects  
4530  [{'nome': 'José Vieira', 'codigo': '25632', 's...  
4531  [{'nome': 'Sra. Yasmin Fernandes', 'codigo': '...  
4532                                                 []  
4533  [{'nome': 'Arthur Almeida', 'codigo': '26338',...  
4534  [{'nome': 'Ana Luiza Vieira', 'codigo': '26361...  


In [8]:
print(df_vagas.head())
print(df_vagas.columns)

                                    informacoes_basicas  \
5185  {'data_requicisao': '04-05-2021', 'limite_espe...   
5184  {'data_requicisao': '04-05-2021', 'limite_espe...   
5183  {'data_requicisao': '04-05-2021', 'limite_espe...   
5182  {'data_requicisao': '04-05-2021', 'limite_espe...   
5181  {'data_requicisao': '04-05-2021', 'limite_espe...   

                                            perfil_vaga  \
5185  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
5184  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
5183  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
5182  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
5181  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   

                                             beneficios  
5185  {'valor_venda': '-', 'valor_compra_1': 'R$', '...  
5184  {'valor_venda': '-', 'valor_compra_1': 'R$', '...  
5183  {'valor_venda': '-', 'valor_compra_1': 'R$', '...  
5182  {'valor_venda': '- p/ mês (168h)', 'valor_comp...  


In [9]:
df_applicants.info()
print(df_applicants.columns)

<class 'pandas.core.frame.DataFrame'>
Index: 42482 entries, 31000 to 5999
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   infos_basicas              42482 non-null  object
 1   informacoes_pessoais       42482 non-null  object
 2   informacoes_profissionais  42482 non-null  object
 3   formacao_e_idiomas         42482 non-null  object
 4   cargo_atual                42482 non-null  object
 5   cv_pt                      42482 non-null  object
 6   cv_en                      42482 non-null  object
dtypes: object(7)
memory usage: 2.6+ MB
Index(['infos_basicas', 'informacoes_pessoais', 'informacoes_profissionais',
       'formacao_e_idiomas', 'cargo_atual', 'cv_pt', 'cv_en'],
      dtype='object')


In [10]:
df_prospects.info()
print(df_prospects.columns)

<class 'pandas.core.frame.DataFrame'>
Index: 14222 entries, 4530 to 14222
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   titulo      14222 non-null  object
 1   modalidade  14222 non-null  object
 2   prospects   14222 non-null  object
dtypes: object(3)
memory usage: 444.4+ KB
Index(['titulo', 'modalidade', 'prospects'], dtype='object')


In [11]:

df_vagas.info()
print(df_vagas.columns)

<class 'pandas.core.frame.DataFrame'>
Index: 14081 entries, 5185 to 12364
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   informacoes_basicas  14081 non-null  object
 1   perfil_vaga          14081 non-null  object
 2   beneficios           14081 non-null  object
dtypes: object(3)
memory usage: 440.0+ KB
Index(['informacoes_basicas', 'perfil_vaga', 'beneficios'], dtype='object')


In [6]:
# 1) Garantir que o índice seja o id da vaga
df_prospects = df_prospects.copy()
df_prospects.index.name = "id_vaga"

# 2) uma linha por candidato
df_px = df_prospects.explode("prospects").reset_index()

# 3) Transformar o dicionário de cada prospect em colunas
df_px = pd.concat(
    [df_px.drop(columns=["prospects"]),
     df_px["prospects"].apply(pd.Series)],
    axis=1
)



In [13]:
print(df_px.columns)
print(df_px.head(2))



Index([           'id_vaga',             'titulo',         'modalidade',
                     'nome',             'codigo', 'situacao_candidado',
         'data_candidatura', 'ultima_atualizacao',         'comentario',
               'recrutador',                    0],
      dtype='object')
  id_vaga               titulo modalidade                      nome codigo  \
0    4530  CONSULTOR CONTROL M                          José Vieira  25632   
1    4530  CONSULTOR CONTROL M             Srta. Isabela Cavalcante  25529   

            situacao_candidado data_candidatura ultima_atualizacao  \
0  Encaminhado ao Requisitante       25-03-2021         25-03-2021   
1  Encaminhado ao Requisitante       22-03-2021         23-03-2021   

                                          comentario         recrutador   0  
0               Encaminhado para  - PJ R$ 72,00/hora  Ana Lívia Moreira NaN  
1  encaminhado para  - R$ 6.000,00 – CLT Full , n...  Ana Lívia Moreira NaN  


In [14]:
print(type(df_px.iloc[0]["titulo"]))
print(type(df_px.iloc[0]["modalidade"]))


<class 'str'>
<class 'str'>


In [15]:
print(df_px.columns.tolist())

['id_vaga', 'titulo', 'modalidade', 'nome', 'codigo', 'situacao_candidado', 'data_candidatura', 'ultima_atualizacao', 'comentario', 'recrutador', 0]


In [7]:
df_px = df_px.copy()

df_px["contratado"] = df_px['situacao_candidado'].apply(
    lambda x: 1 if str(x).lower().strip() == "encaminhado ao requisitante" else 0
)

df_px["contratado"].value_counts()


contratado
0    40580
1    16122
Name: count, dtype: int64

In [8]:
df_applicants.info()
print(df_applicants.columns)
print(df_applicants.head())

<class 'pandas.core.frame.DataFrame'>
Index: 42482 entries, 31000 to 5999
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   infos_basicas              42482 non-null  object
 1   informacoes_pessoais       42482 non-null  object
 2   informacoes_profissionais  42482 non-null  object
 3   formacao_e_idiomas         42482 non-null  object
 4   cargo_atual                42482 non-null  object
 5   cv_pt                      42482 non-null  object
 6   cv_en                      42482 non-null  object
dtypes: object(7)
memory usage: 2.6+ MB
Index(['infos_basicas', 'informacoes_pessoais', 'informacoes_profissionais',
       'formacao_e_idiomas', 'cargo_atual', 'cv_pt', 'cv_en'],
      dtype='object')
                                           infos_basicas  \
31000  {'telefone_recado': '', 'telefone': '(11) 9704...   
31001  {'telefone_recado': '', 'telefone': '(11) 9372...   
31002  {'telefone

In [9]:

dados_contatos = df_applicants["informacoes_pessoais"].dropna().iloc[0]
print(dados_contatos.keys())


dict_keys(['data_aceite', 'nome', 'cpf', 'fonte_indicacao', 'email', 'email_secundario', 'data_nascimento', 'telefone_celular', 'telefone_recado', 'sexo', 'estado_civil', 'pcd', 'endereco', 'skype', 'url_linkedin', 'facebook'])


In [10]:
# Preparar base de candidatos
df_app = df_applicants.copy()
df_app.index.name = "codigo"
df_app = df_app.reset_index()

# garantir tipo do codigo (pra bater com prospects)
df_app["codigo"] = pd.to_numeric(df_app["codigo"], errors="coerce")

# Abrir dicionários principais (mantendo seus nomes)
app_bas = df_app["informacoes_pessoais"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("cand_")

app_form = df_app["formacao_e_idiomas"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("form_")

# NOVOS: infos_basicas e informacoes_profissionais
app_info = df_app["infos_basicas"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("info_")

app_prof = df_app["informacoes_profissionais"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("prof_")

# Dataset final de candidatos 
df_app_flat = pd.concat(
    [
        df_app[["codigo", "cargo_atual"]],
        app_bas,
        app_form,
        app_info,   
        app_prof  
    ],
    axis=1
)

print(df_app_flat.shape)
df_app_flat.head(3)


(42482, 47)


,codigo,cargo_atual,cand_data_aceite,cand_nome,cand_cpf,cand_fonte_indicacao,cand_email,cand_email_secundario,cand_data_nascimento,cand_telefone_celular,...,info_nome,prof_titulo_profissional,prof_area_atuacao,prof_conhecimentos_tecnicos,prof_certificacoes,prof_outras_certificacoes,prof_remuneracao,prof_nivel_profissional,prof_qualificacoes,prof_experiencias
0,31000,{},Cadastro anterior ao registro de aceite,Carolina Aparecida,,:,carolina_aparecida@gmail.com,,0000-00-00,(11) 97048-2708,...,Carolina Aparecida,,,,,,,,NaN,NaN
1,31001,{},Cadastro anterior ao registro de aceite,Eduardo Rios,,Outros: Contato do RH,eduardo_rios@hotmail.com,,28-12-1994,(11) 93723-4396,...,Eduardo Rios,Analista Administrativo,Administrativa,,,,1900,,NaN,NaN
2,31002,{},Cadastro anterior ao registro de aceite,Pedro Henrique Carvalho,,Anúncio:,pedro_henrique_carvalho@gmail.com,,12-12-1988,(11) 92399-9824,...,Pedro Henrique Carvalho,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,"2.500,00",,NaN,NaN


In [11]:

df_v = df_vagas.copy()
df_v.index.name = "id_vaga"
df_v = df_v.reset_index()

vaga_bas = df_v["informacoes_basicas"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("vaga_")
vaga_perf = df_v["perfil_vaga"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("perfil_")
vaga_ben  = df_v["beneficios"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("ben_")

df_v_flat = pd.concat(
    [df_v[["id_vaga"]], vaga_bas, vaga_perf, vaga_ben],
    axis=1
)

df_v_flat["id_vaga"] = pd.to_numeric(df_v_flat["id_vaga"], errors="coerce")
print(df_v_flat.shape)


(14081, 45)


In [12]:
df_app = df_applicants.copy()
df_app.index.name = "codigo"
df_app = df_app.reset_index()
df_app["codigo"] = pd.to_numeric(df_app["codigo"], errors="coerce")

app_bas  = df_app["informacoes_pessoais"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("cand_")

# padroniza o nome do telefone para ficar mais fácil no resto do projeto
if "cand_telefone_celular" in app_bas.columns:
    app_bas = app_bas.rename(columns={"cand_telefone_celular": "cand_telefone"})

app_form = df_app["formacao_e_idiomas"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("form_")

# novos blocos (sem quebrar seus nomes)
app_info = df_app["infos_basicas"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("info_")
app_prof = df_app["informacoes_profissionais"].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).add_prefix("prof_")

df_app_flat = pd.concat(
    [
        df_app[["codigo", "cargo_atual"]],
        app_bas,
        app_form,
        app_info,
        app_prof
    ],
    axis=1
)

print(df_app_flat.shape)


(42482, 47)


In [13]:
# prospects
df_prospects = df_prospects.copy()
df_prospects.index.name = "id_vaga"

df_px = df_prospects.explode("prospects").reset_index()
df_px = df_px[df_px["prospects"].notna()].copy()

df_px = pd.concat(
    [df_px.drop(columns=["prospects"]),
     df_px["prospects"].apply(pd.Series)],
    axis=1
)

# padronizar chaves (importante para o merge)
df_px["codigo"] = pd.to_numeric(df_px["codigo"], errors="coerce")
df_px["id_vaga"] = pd.to_numeric(df_px["id_vaga"], errors="coerce")

df_app_flat["codigo"] = pd.to_numeric(df_app_flat["codigo"], errors="coerce")
df_v_flat["id_vaga"] = pd.to_numeric(df_v_flat["id_vaga"], errors="coerce")

# remover linhas sem chave (senão merge vira bagunça)
df_px = df_px[df_px["codigo"].notna() & df_px["id_vaga"].notna()].copy()

# JOIN com candidatos
df_ml = df_px.merge(df_app_flat, on="codigo", how="left")

# JOIN com vagas
df_ml = df_ml.merge(df_v_flat, on="id_vaga", how="left")

print(df_ml.shape)

# checagens simples para entender se está batendo
print("(cand_nome nulo):", df_ml["cand_nome"].isna().sum() if "cand_nome" in df_ml.columns else "cand_nome não existe")
print("vagas_nomes incompletos ", df_ml.filter(like="vaga_").isna().all(axis=1).sum() if len(df_ml.filter(like="vaga_").columns) > 0 else "colunas vaga_ não existem")



(53759, 100)
(cand_nome nulo): 8664
vagas_nomes incompletos  24


In [14]:
df_ml["contratado"] = (
    df_ml["situacao_candidado"].astype(str).str.lower().str.strip()
      .eq("encaminhado ao requisitante")
      .astype(int)
)

df_ml_clean = df_ml[
    df_ml["codigo"].notna() &
    df_ml["cand_nome"].notna()
].copy()

def limpar_texto(s):
    s = "" if s is None else str(s)
    s = s.lower()
    s = re.sub(r"\b(dr|dra|doutor|doutora|phd|ph\.d)\b", " ", s)
    s = re.sub(r"[^a-zà-ÿ0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def col_texto(df, col):
    if col in df.columns:
        return df[col].fillna("").astype(str)
    else:
        return pd.Series([""] * len(df), index=df.index)

def col_texto_dict(df, col):
    if col in df.columns:
        return df[col].apply(lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, (dict, list)) else "")
    else:
        return pd.Series([""] * len(df), index=df.index)

# junta colunas por prefixo e vira texto
def juntar_colunas_por_prefixo(df, prefixo):
    cols = [c for c in df.columns if c.startswith(prefixo)]
    if not cols:
        return pd.Series([""] * len(df), index=df.index)
    return (
        df[cols]
        .fillna("")
        .astype(str)
        .agg(" ".join, axis=1)
    )

# TEXTO DO CANDIDATO (o que ele é / sabe)
df_ml_clean["cand_texto"] = (
    df_ml_clean["cargo_atual"].fillna("").astype(str) + " " +
    juntar_colunas_por_prefixo(df_ml_clean, "prof_") + " " +
    juntar_colunas_por_prefixo(df_ml_clean, "form_") + " " +
    juntar_colunas_por_prefixo(df_ml_clean, "cand_")
).apply(limpar_texto)

# TEXTO DA VAGA (o que a vaga pede)
df_ml_clean["vaga_texto"] = (
    juntar_colunas_por_prefixo(df_ml_clean, "perfil_") + " " +
    juntar_colunas_por_prefixo(df_ml_clean, "vaga_") + " " +
    juntar_colunas_por_prefixo(df_ml_clean, "ben_")
).apply(limpar_texto)


In [24]:
print("linhas:", len(df_ml))
print("codigo nulo:", df_ml["codigo"].isna().sum())
print("cand_nome nulo:", df_ml["cand_nome"].isna().sum() if "cand_nome" in df_ml.columns else "sem cand_nome")
print("id_vaga nulo:", df_ml["id_vaga"].isna().sum())
print("titulo(prospect) nulo:", df_ml["titulo"].isna().sum())
print("modalidade(prospect) nulo:", df_ml["modalidade"].isna().sum())


linhas: 53759
codigo nulo: 0
cand_nome nulo: 8664
id_vaga nulo: 0
titulo(prospect) nulo: 0
modalidade(prospect) nulo: 0


In [15]:
# remove candidatos sem nome (não fazem sentido para ranking)
df_ml_clean = df_ml_clean[df_ml_clean["cand_nome"].notna()].copy()

print("Linhas após remover cand_nome nulo:", len(df_ml_clean))


Linhas após remover cand_nome nulo: 45095


In [26]:
df_ml_clean["cand_nome"].isna().sum()


np.int64(0)

In [16]:
# junta todos os textos (candidatos + vagas)
corpus = pd.concat(
    [df_ml_clean["cand_texto"], df_ml_clean["vaga_texto"]],
    axis=0
).fillna("")

# cria o modelo TF-IDF
tfidf = TfidfVectorizer(
    min_df=3,          # ignora palavras muito raras
    ngram_range=(1,2)  # usa palavras e pares de palavras
)

# aprende o vocabulário
tfidf.fit(corpus)


# transforma texto em vetores
V_cand = normalize(tfidf.transform(df_ml_clean["cand_texto"].fillna("")))
V_vaga = normalize(tfidf.transform(df_ml_clean["vaga_texto"].fillna("")))

# similaridade cosseno (linha a linha)
df_ml_clean["sim_texto"] = (V_cand.multiply(V_vaga)).sum(axis=1).A1


In [17]:
df_hist = df_ml_clean.copy()

total_apps = df_hist.groupby("codigo")["id_vaga"].size().rename("qtd_aplicacoes_total")
distinct_vagas = df_hist.groupby("codigo")["id_vaga"].nunique().rename("qtd_vagas_distintas_total")

occ_cand_vaga = (
    df_hist.groupby(["codigo", "id_vaga"]).size()
    .rename("occ_cand_vaga")
    .reset_index()
)

df_hist = df_hist.merge(total_apps, on="codigo", how="left")
df_hist = df_hist.merge(distinct_vagas, on="codigo", how="left")
df_hist = df_hist.merge(occ_cand_vaga, on=["codigo", "id_vaga"], how="left")

df_hist["qtd_aplicacoes"] = (df_hist["qtd_aplicacoes_total"] - 1).clip(lower=0)
reduz_distinto = (df_hist["occ_cand_vaga"] == 1).astype(int)
df_hist["qtd_vagas_distintas"] = (df_hist["qtd_vagas_distintas_total"] - reduz_distinto).clip(lower=0)

cand_mode = (
    df_hist.groupby("codigo")
    .agg(
        modalidade_mais_freq=("modalidade", lambda s: s.astype(str).value_counts().index[0] if len(s.dropna()) else "Desconhecido"),
        titulo_mais_freq=("titulo", lambda s: s.astype(str).value_counts().index[0] if len(s.dropna()) else "Desconhecido"),
    )
    .reset_index()
)

df_ml2 = df_hist.merge(cand_mode, on="codigo", how="left")


In [18]:
y = df_ml2["contratado"].astype(int)
df_ml2["sim_texto"] = df_ml_clean["sim_texto"].values

X = df_ml2[[
    "modalidade",
    "titulo",
    "qtd_aplicacoes",
    "qtd_vagas_distintas",
    "modalidade_mais_freq",
    "titulo_mais_freq",
     "sim_texto"
]].copy()

# nulos e tipos
X["qtd_aplicacoes"] = pd.to_numeric(X["qtd_aplicacoes"], errors="coerce").fillna(0)
X["qtd_vagas_distintas"] = pd.to_numeric(X["qtd_vagas_distintas"], errors="coerce").fillna(0)

X = X.fillna("Desconhecido")


In [19]:
X["sim_texto"] = pd.to_numeric(X["sim_texto"], errors="coerce").fillna(0)

In [20]:
TOPK = 500

top_titulos = X["titulo"].value_counts().head(TOPK).index.tolist()
X["titulo"] = X["titulo"].where(X["titulo"].isin(top_titulos), "Outros")

top_titulos2 = X["titulo_mais_freq"].value_counts().head(TOPK).index.tolist()
X["titulo_mais_freq"] = X["titulo_mais_freq"].where(X["titulo_mais_freq"].isin(top_titulos2), "Outros")

X_enc = pd.get_dummies(X, drop_first=True)
print("X_enc:", X_enc.shape)


X_enc: (45095, 1013)


In [32]:
print("Qtd Outros:", (X["titulo"] == "Outros").sum())
print("Qtd total:", len(X))
print("Percentual Outros:", ((X["titulo"] == "Outros").mean() * 100).round(2), "%")


Qtd Outros: 32201
Qtd total: 45095
Percentual Outros: 71.41 %


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    X_enc, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1
)

model.fit(X_train, y_train)

pred = model.predict(X_test)

print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))



              precision    recall  f1-score   support

           0       0.84      0.39      0.53      6257
           1       0.37      0.83      0.51      2762

    accuracy                           0.52      9019
   macro avg       0.60      0.61      0.52      9019
weighted avg       0.69      0.52      0.52      9019

[[2421 3836]
 [ 476 2286]]


In [22]:
X.isna().sum().sort_values(ascending=False).head(10)

modalidade              0
titulo                  0
qtd_aplicacoes          0
qtd_vagas_distintas     0
modalidade_mais_freq    0
titulo_mais_freq        0
sim_texto               0
dtype: int64

In [23]:
from pathlib import Path
import joblib

ART = Path.cwd() / "artifacts"
ART.mkdir(exist_ok=True)

joblib.dump(model, ART / "modelo_match.pkl")
joblib.dump(X_enc.columns.tolist(), ART / "features_match.pkl")
joblib.dump(top_titulos, ART / "top_titulos.pkl")
joblib.dump(top_titulos2, ART / "top_titulos2.pkl")
joblib.dump(tfidf, ART / "tfidf_match.pkl")

df_v_flat.to_parquet(ART / "vagas_flat.parquet", index=False)
df_ml2.to_parquet(ART / "df_ml2.parquet", index=False)


print("Artefatos salvos em /artifacts")

Artefatos salvos em /artifacts
